In [102]:
# Wait 60 seconds before connecting using these details, or login to https://console.neo4j.io to validate the Aura Instance is available
NEO4J_URI="neo4j+s://f2a9c169.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="wakSwADnPJZxg-Lyq3wEr_aAkPCdcM1H3Bkqc5njEoY"
AURA_INSTANCEID="f2a9c169"
AURA_INSTANCENAME="Instance01"


In [103]:
from langchain_community.graphs import Neo4jGraph


In [105]:
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)


In [106]:
from langchain.document_loaders import WikipediaLoader


In [107]:

docs = WikipediaLoader(query="Elizabeth I").load()

c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [108]:
len(docs)


23

In [109]:
docs[0].page_content



'Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.\nElizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents\' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry\'s death in 1547, Elizabeth\'s younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward\'s will was set aside within weeks of his death and Mary became queen, deposing and executing Jane. During Mary\'s reign, Elizabeth was imprisoned for nearly a year on suspicion of supporting 

In [110]:
from langchain.llms import Ollama

In [111]:
model = Ollama(model="llama3")


In [112]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [113]:
content = docs[0].page_content

In [114]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=300 
)


In [115]:
chunks = text_splitter.split_text(content)

In [31]:
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:")
    print(chunk)
    print("\n" + "-"*80 + "\n")

Chunk 1:
Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.

--------------------------------------------------------------------------------

Chunk 2:
Elizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry's death in 1547, Elizabeth's younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward's will was set aside within weeks of his death and Mary became queen, deposing and executing Jan

In [116]:
from langchain.schema import Document

In [54]:
from pydantic import ValidationError


In [55]:
documents = [Document(page_content=chunk) for chunk in chunks]

In [56]:
from langchain_experimental.graph_transformers import LLMGraphTransformer


In [57]:
llm_transformer = LLMGraphTransformer(llm=model)

In [58]:
try:
    graph_documents = llm_transformer.convert_to_graph_documents(documents)
    # Print or use graph_documents
    for gd in graph_documents:
        print(gd)
except AttributeError as e:
    print(f"Attribute error: {e}")
except ValidationError as e:
    print(f"Validation error: {e}")

nodes=[Node(id='Elizabeth I', type='Person'), Node(id='England and Ireland', type='Location'), Node(id='17 November 1558', type='Date'), Node(id='1603', type='Date')] relationships=[Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='England and Ireland', type='Location'), type='WAS_QUEEN_OF'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='1603', type='Date'), type='DIED_IN'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='17 November 1558', type='Date'), type='REIGNED_FROM')] source=Document(page_content='Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.')
nodes=[Node(id='Mary', type='Person'), Node(id='Anne Boleyn', type='Person'), Node(id='Edward VI', type='Person'), Node(id='Lady Jane Grey', type='Person'), Node(id='Henry VIII', type='Person'), Node(id='Elizabeth', type='Person'),

In [59]:
graph_documents

[GraphDocument(nodes=[Node(id='Elizabeth I', type='Person'), Node(id='England and Ireland', type='Location'), Node(id='17 November 1558', type='Date'), Node(id='1603', type='Date')], relationships=[Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='England and Ireland', type='Location'), type='WAS_QUEEN_OF'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='1603', type='Date'), type='DIED_IN'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='17 November 1558', type='Date'), type='REIGNED_FROM')], source=Document(page_content='Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.')),
 GraphDocument(nodes=[Node(id='Mary', type='Person'), Node(id='Anne Boleyn', type='Person'), Node(id='Edward VI', type='Person'), Node(id='Lady Jane Grey', type='Person'), Node(id='Henry VIII', type='Person'), No

In [22]:
# graph.add_graph_documents(graph_documents)

In [32]:
for document in graph_documents:
    print("Document:", document)
    for node in document.nodes:
        print("Node:", node)
        print("Node Labels:", getattr(node, 'labels', 'No labels attribute'))
        print("Node Properties:", node.__dict__)


NameError: name 'graph_documents' is not defined

In [61]:
def clean_node_labels(node):
    """Clean and validate node labels."""
    if hasattr(node, 'labels'):
        # Clean the labels by removing None or empty string values
        node.labels = [label for label in node.labels if label]
    return node


In [62]:
def validate_node(node):
    """Check if a node is valid by ensuring it has necessary attributes."""
    return bool(node.id and node.type)

def sanitize_graph_documents(documents):
    sanitized_docs = []
    for doc in documents:
        sanitized_nodes = [clean_node_labels(node) for node in doc.nodes if validate_node(node)]
        sanitized_relationships = [
            rel for rel in doc.relationships
            if validate_node(rel.source) and validate_node(rel.target)
        ]
        page_content = getattr(doc.source, 'page_content', '')
        sanitized_docs.append(Document(
            nodes=sanitized_nodes,
            relationships=sanitized_relationships,
            source=doc.source,
            page_content=page_content
        ))
    return sanitized_docs


In [63]:
class Document:
    def __init__(self, nodes, relationships, source, page_content):
        self.nodes = nodes
        self.relationships = relationships
        self.source = source
        self.page_content = page_content


In [64]:
def get_page_content(source):
    # Implement this based on your source structure
    return getattr(source, 'page_content', '')

def sanitize_graph_documents(documents):
    sanitized_docs = []
    for doc in documents:
        sanitized_nodes = [clean_node_labels(node) for node in doc.nodes if validate_node(node)]
        sanitized_relationships = [
            rel for rel in doc.relationships
            if validate_node(rel.source) and validate_node(rel.target)
        ]
        page_content = get_page_content(doc.source)
        sanitized_docs.append(Document(
            nodes=sanitized_nodes,
            relationships=sanitized_relationships,
            source=doc.source,
            page_content=page_content
        ))
    return sanitized_docs


In [66]:
try:
    sanitized_graph_documents = sanitize_graph_documents(graph_documents)
    graph.add_graph_documents(sanitized_graph_documents)
except Exception as e:
    print(f"Error: {e}")


In [117]:
import json
import re
from langchain.llms import Ollama

model = Ollama(model="llama3")

def identify_entities(query):
    prompt = (
        f"Identify all entities (people, organizations) in the following text:\n\n"
        f"{query}\n\n"
        "Return only the entity names as a JSON array. Example format:\n"
        '["Entity1", "Entity2", "Entity3"]'
    )
    
    response = model.generate([prompt])
    
    response_text = response.generations[0][0].text.strip()
    
    print("Raw model response:", response_text)
    
    json_text = re.search(r'\[.*?\]', response_text, re.DOTALL)
    
    if json_text:
        json_text = json_text.group()
        try:
            entities_json = json.loads(json_text)
        except json.JSONDecodeError:
            print("Error decoding JSON from the extracted text.")
            entities_json = []
    else:
        print("No JSON array found in the response.")
        entities_json = []
    
    return entities_json



In [118]:
def find_all_relations(entities):
    relations = []

    for entity in entities:
        query = (
            f"MATCH (e)-[r]-(related) "
            f"WHERE e.id = '{entity}' "
            f"RETURN e.id AS entity, type(r) AS relationship, related.id AS related_entity"
        )

        try:
            result = graph.query(query)
            for record in result:
                relations.append({
                    "entity": record["entity"],
                    "relationship": record["relationship"],
                    "related_entity": record["related_entity"]
                })
        except Exception as e:
            print(f"Error executing query for entity '{entity}': {e}")

    return relations


In [119]:

def display_relations_as_text(relations):
    queries = []
    for relation in relations:
        entity = relation['entity']
        relationship = relation['relationship']
        related_entity = relation['related_entity']
        queries.append(f"{entity} -> {relationship} -> {related_entity}")
    
    # Combine all relations into a single query string
    query_string = " . ".join(queries) + " . "
    return query_string

In [120]:
from langchain.embeddings import HuggingFaceEmbeddings


In [121]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Bot

In [122]:
from langchain_community.vectorstores import Neo4jVector


In [123]:
db = Neo4jVector.from_documents(
    docs, embedding_model, url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD
)

In [40]:
docs_with_score = db.similarity_search_with_score(query, k=2)

In [41]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.7943490147590637
Elizabeth Stuart (19 August 1596 – 13 February 1662) was Electress of the Palatinate and briefly Queen of Bohemia as the wife of Frederick V of the Palatinate. The couple's selection for the crown by the nobles of Bohemia was part of the political and religious turmoil setting off the Thirty Years' War. Since her husband's reign in Bohemia lasted over one winter, she is called "the Winter Queen" (German: Die Winterkönigin, Czech: Zimní královna).
Princess Elizabeth was the only surviving daughter of James VI and I, King of Scotland, England, and Ireland, and his queen, Anne of Denmark; she was the elder sister of Charles I. Born in Scotland, she was named in honour of her father's predecessor and cousin in England, Elizabeth I.  During Elizabeth Stuart's childhood, unbeknownst to her, part of the failed Gunpowder Plot was a scheme to replace her father with her on the throne, and

In [124]:
from langchain.prompts import PromptTemplate

template = """
You are an AI trained to analyze and interpret text data. Given the following query and the related documents, please provide a detailed and insightful response.
give concise answer. I give you multiple documents from my knowledege base and relations from graph give concise answer related to my context whhich i pick from db . dont mention based on provided query only give summary of my documets and relation accordig to my query
dont mention that info is retrieved from that document. if there is no related information of query in relation and documents directly say i dont know. else proceed if founf any details.
Query: {query}

Related Documents:
{docs}

Relations:
{relations}

Response:
"""
prompt_template = PromptTemplate(
    input_variables=["query", "docs", "relations"],
    template=template
)


In [125]:

query = "which house did Elizabeth I belongs to?"
# Fetch similarity search results
entitites = identify_entities(query)

docs_with_score = db.similarity_search_with_score(query, k=2)

summaries = [doc[0].metadata['summary'] for doc in docs_with_score]


relations = find_all_relations(entitites)
# Define relations as text

relations_text = display_relations_as_text(relations)

# Format the prompt input
prompt_input = prompt_template.format(
    query=query,
    docs=summaries,
    relations=relations_text
)


Raw model response: Here is the list of entities mentioned in the text:

["Elizabeth I"]

Note: There is only one person mentioned in the text, which is Queen Elizabeth I.


In [126]:
entitites

['Elizabeth I']

In [127]:
summaries

['Elizabeth Stuart (19 August 1596 – 13 February 1662) was Electress of the Palatinate and briefly Queen of Bohemia as the wife of Frederick V of the Palatinate. The couple\'s selection for the crown by the nobles of Bohemia was part of the political and religious turmoil setting off the Thirty Years\' War. Since her husband\'s reign in Bohemia lasted over one winter, she is called "the Winter Queen" (German: Die Winterkönigin, Czech: Zimní královna).\nPrincess Elizabeth was the only surviving daughter of James VI and I, King of Scotland, England, and Ireland, and his queen, Anne of Denmark; she was the elder sister of Charles I. Born in Scotland, she was named in honour of her father\'s predecessor and cousin in England, Elizabeth I.  During Elizabeth Stuart\'s childhood, unbeknownst to her, part of the failed Gunpowder Plot was a scheme to replace her father with her on the throne, and forcibly raise her as a Catholic.  \nHer father later arranged for her marriage to the Protestant F

In [128]:
docs_with_score

[(Document(metadata={'summary': 'Elizabeth Stuart (19 August 1596 – 13 February 1662) was Electress of the Palatinate and briefly Queen of Bohemia as the wife of Frederick V of the Palatinate. The couple\'s selection for the crown by the nobles of Bohemia was part of the political and religious turmoil setting off the Thirty Years\' War. Since her husband\'s reign in Bohemia lasted over one winter, she is called "the Winter Queen" (German: Die Winterkönigin, Czech: Zimní královna).\nPrincess Elizabeth was the only surviving daughter of James VI and I, King of Scotland, England, and Ireland, and his queen, Anne of Denmark; she was the elder sister of Charles I. Born in Scotland, she was named in honour of her father\'s predecessor and cousin in England, Elizabeth I.  During Elizabeth Stuart\'s childhood, unbeknownst to her, part of the failed Gunpowder Plot was a scheme to replace her father with her on the throne, and forcibly raise her as a Catholic.  \nHer father later arranged for h

In [130]:
relations_text

'Elizabeth I -> WAS_QUEEN_OF -> England and Ireland . Elizabeth I -> DIED_IN -> 1603 . Elizabeth I -> REIGNED_FROM -> 17 November 1558 . '

In [131]:
response = model(prompt_input)

c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [132]:
print(response)

Elizabeth I belonged to the House of Tudor.


In [ ]:
# complete